In [3]:
#import plugins
import biom
import os
import pandas as pd
import qiime2 as q2
from qiime2.plugins.emperor.actions import plot, biplot
from qiime2.plugins.deicode.actions import rpca
from qiime2.plugins.diversity.actions import core_metrics_phylogenetic
from qiime2.plugins.fragment_insertion.methods import filter_features
from qiime2.plugins.diversity.visualizers import beta_group_significance
from qiime2.plugins.diversity.visualizers import alpha_group_significance
from qiime2.plugins.metadata.visualizers import tabulate
from qiime2.plugins.diversity.actions import alpha #I replaced "methods" with "actions"
from qiime2.plugins import taxa

In [4]:
SEQ_TYPE='shotgun'

In [10]:
#import filtered table 
#tree_filtered_table = q2.Artifact.load('{SEQ_TYPE}/filtered-table.qza'.format(SEQ_TYPE=SEQ_TYPE))

#import filtered metadata
q2mf_filtered = q2.Metadata.load('HBM_Matrix_LCMS_1500/metadata_ms_HBM.tsv'.format(SEQ_TYPE=SEQ_TYPE))

#import tree
#rooted_tree = q2.Artifact.load('{SEQ_TYPE}/tree.qza'.format(SEQ_TYPE=SEQ_TYPE))

In [6]:
#core diversity metrics @ different sequencing depths
# for rf_depth in [1000, 10000, 100000, 500000, 1000000 ]:
#     core_results = core_metrics_phylogenetic(tree_filtered_table,
#                                              rooted_tree,
#                                              rf_depth,
#                                              q2mf_filtered)
    
#     os.mkdir(('{SEQ_TYPE}/core-metrics-%i'.format(SEQ_TYPE=SEQ_TYPE))%(rf_depth))
    
#     for name, q2art in zip(core_results._fields, core_results):
#         out_ = os.path.join(('{SEQ_TYPE}/core-metrics-%i'.format(SEQ_TYPE=SEQ_TYPE))%(rf_depth),name)
#         q2art.save(out_)

/Users/caitrionabrennan/opt/anaconda3/envs/qiime2-2021.8/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:1776: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/Users/caitrionabrennan/opt/anaconda3/envs/qiime2-2021.8/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.016203284901642002 and the largest is 17.006482277512927.
  warn(
/Users/caitrionabrennan/opt/anaconda3/envs/qiime2-2021.8/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eig

FileExistsError: [Errno 17] File exists: 'shotgun/core-metrics-1000'

In [ ]:
#visualize core metric qzv plots in jupyter notebooks
q2.Visualization.load('{SEQ_TYPE}/core-metrics-1000/unweighted_unifrac_emperor.qzv'.format(SEQ_TYPE=SEQ_TYPE))

In [ ]:
#visualize core metric qzv plots in jupyter notebooks
q2.Visualization.load('{SEQ_TYPE}/core-metrics-500000/weighted_unifrac_emperor.qzv'.format(SEQ_TYPE=SEQ_TYPE))

In [ ]:
#make deicode/rpca beta diversity plot
rpcares = rpca(tree_filtered_table)

#makes a directory to put the output in
os.mkdir('{SEQ_TYPE}/rpca-res-fresh'.format(SEQ_TYPE=SEQ_TYPE))
for name, q2art in zip(rpcares._fields, rpcares):
    out_ = os.path.join('{SEQ_TYPE}/rpca-res-fresh'.format(SEQ_TYPE=SEQ_TYPE),name)
    q2art.save(out_)

In [ ]:
#import biplot made by rpca
bi_plot = q2.Artifact.load('{SEQ_TYPE}/rpca-res-fresh/biplot.qza'.format(SEQ_TYPE=SEQ_TYPE))
bi_plot

In [ ]:
empbiplt = biplot(bi_plot,q2mf_filtered)
empbiplt

In [ ]:
empbiplt.visualization

In [ ]:
#save rpca visualization
empbiplt.visualization.save('{SEQ_TYPE}/rpca-res-fresh/empbiplt.qzv'.format(SEQ_TYPE=SEQ_TYPE))

In [ ]:
UnUF_dist_matx = q2.Artifact.load(('{SEQ_TYPE}/core-metrics-%i/unweighted_unifrac_distance_matrix.qza'.format(SEQ_TYPE=SEQ_TYPE))%(rf_depth))
UnUF_subject = beta_group_significance(UnUF_dist_matx,q2mf_filtered.get_column('replicate_id'))
UnUF_subject.visualization.save(('{SEQ_TYPE}/core-metrics-%i/UnUF_replicate.qzv'.format(SEQ_TYPE=SEQ_TYPE))%(rf_depth))

In [ ]:
for rf_depth in [1000, 10000, 100000, 500000, 1000000 ]:
    UnUF_dist_matx = q2.Artifact.load(('{SEQ_TYPE}/core-metrics-%i/weighted_unifrac_distance_matrix.qza'.format(SEQ_TYPE=SEQ_TYPE))%(rf_depth))
    UnUF_fresh = beta_group_significance(UnUF_dist_matx,q2mf_filtered.get_column('storage_solution'))
    UnUF_fresh.visualization.save(('{SEQ_TYPE}/core-metrics-%i/UnUF_storage.qzv'.format(SEQ_TYPE=SEQ_TYPE))%(rf_depth))
    
    UnUF_fresh = beta_group_significance(UnUF_dist_matx,q2mf_filtered.get_column('extraction_protocol'))
    UnUF_fresh.visualization.save(('{SEQ_TYPE}/core-metrics-%i/UnUF_extract.qzv'.format(SEQ_TYPE=SEQ_TYPE))%(rf_depth))
    #F stat 1.5829
    #pvalue 0.11

In [ ]:
UnUF_dist_matx = q2.Artifact.load('shotgun/core-metrics-500000/unweighted_unifrac_distance_matrix.qza')
UnUF_fresh = beta_group_significance(UnUF_dist_matx,q2mf_filtered.get_column('extraction_protocol'))
UnUF_fresh.visualization.save('shotgun/core-metrics-500000/UnUF_extract.qzv')
#F stat 1.5829
#pvalue 0.11

In [ ]:
#beta diversity significance weighted unifrac 10000
WeUF_dist_matx = q2.Artifact.load('core-metrics-10000/weighted_unifrac_distance_matrix.qza')
WeUF_species = beta_group_significance(WeUF_dist_matx,q2mf_filtered.get_column('host_common_name'))
#save beta diversity significance unweighted unifrac 10000 visualization
WeUF_species.visualization.save('core-metrics-10000/WeUF_species.qzv')
#F stat 242.935
#pvalue 0.001

In [ ]:
#beta diversity significance weighted unifrac 10000
WeUF_dist_matx = q2.Artifact.load('core-metrics-1000/weighted_unifrac_distance_matrix.qza')
WeUF_subject = beta_group_significance(WeUF_dist_matx,q2mf_filtered.get_column('host_subject_id'))
#save beta diversity significance unweighted unifrac 10000 visualization
WeUF_subject.visualization.save('core-metrics-1000/WeUF_subject.qzv')
#F stat 123.591
#pvalue 0.001

In [ ]:
#beta diversity significance weighted unifrac 10000
WeUF_dist_matx = q2.Artifact.load('core-metrics-1000/weighted_unifrac_distance_matrix.qza')
WeUF_fresh = beta_group_significance(WeUF_dist_matx,q2mf_filtered.get_column('storage_solution'))
#save beta diversity significance unweighted unifrac 10000 visualization
WeUF_fresh.visualization.save('core-metrics-1000/WeUF_storage.qzv')
#F stat 29.1233
#pvalue 0.001

In [ ]:
#beta diversity significance weighted unifrac 10000
WeUF_dist_matx = q2.Artifact.load('core-metrics-1000/weighted_unifrac_distance_matrix.qza')
WeUF_fresh = beta_group_significance(WeUF_dist_matx,q2mf_filtered.get_column('extraction_protocol'))
#save beta diversity significance unweighted unifrac 10000 visualization
WeUF_fresh.visualization.save('core-metrics-1000/WeUF_extract.qzv')
#F stat 29.1233
#pvalue 0.001

In [8]:
meta = q2.Metadata.load('HBM_Matrix_LCMS_1500/metadata_ms_HBM.tsv'.format(SEQ_TYPE=SEQ_TYPE))
# tax = q2.Artifact.load('{SEQ_TYPE}/taxa.qza'.format(SEQ_TYPE=SEQ_TYPE))
# tax_bp = taxa.actions.barplot(tree_filtered_table, tax, meta)
# tax_bp.visualization

In [ ]:
#beta diversity significance rpca
rpca_dist_matx = q2.Artifact.load('rpca-res/distance_matrix.qza')
rpca_species = beta_group_significance(rpca_dist_matx,q2mf_filtered.get_column('host_common_name'))

In [ ]:
rpca_species.visualization

In [ ]:
#beta diversity significance rpca
rpca_dist_matx = q2.Artifact.load('rpca-res-fresh/distance_matrix.qza')
rpca_species = beta_group_significance(rpca_dist_matx,q2mf_filtered.get_column('fresh_or_not'))
rpca_species.visualization

In [ ]:
#save beta diversity significance rpca
rpca_species.visualization.save('rpca-res/rpca_species.qzv')

In [ ]:
#beta diversity significance rpca
rpca_dist_matx = q2.Artifact.load('rpca-res/distance_matrix.qza')
rpca_species = beta_group_significance(rpca_dist_matx,q2mf_filtered.get_column('host_subject_id'))
rpca_species.visualization.save('rpca-res/rpca_subject.qzv')

In [11]:
#shannon diversity significance and figure (API)
shannon_vector = q2.Artifact.load('HBM_Matrix_LCMS_1500/1500_shannon_matrix_val1_noise_detect2_HBM_R.qza')
shannon_figure = alpha_group_significance(shannon_vector,q2mf_filtered)

In [12]:
#save shannon alpha diversity significance
shannon_figure.visualization.save('HBM_Matrix_LCMS_1500/shannon_figure.qzv')

'HBM_Matrix_LCMS_1500/shannon_figure.qzv'

In [18]:
#beta diversity significance jaccard
j_dm = q2.Artifact.load('HBM_Matrix_LCMS_1500/distance_matrix1500_jaccard_qiime2_noise_detect2_noblanks_HBM_R.qza')
j_stats = beta_group_significance(j_dm,q2mf_filtered.get_column('ATTRIBUTE_host_subject_id'))
j_stats.visualization.save('HBM_Matrix_LCMS_1500/jacc_stats_host.qzv')

'HBM_Matrix_LCMS_1500/jacc_stats_host.qzv'

In [16]:
#beta diversity significance jaccard
j_dm = q2.Artifact.load('HBM_Matrix_LCMS_1500/distance_matrix1500_jaccard_qiime2_noise_detect2_noblanks_HBM_R.qza')
j_stats = beta_group_significance(j_dm,q2mf_filtered.get_column('ATTRIBUTE_storagebuffer'))
j_stats.visualization.save('HBM_Matrix_LCMS_1500/jacc_stats_storage.qzv')

'HBM_Matrix_LCMS_1500/jacc_stats_storage.qzv'

In [ ]:
#beta diversity significance jaccard
j_dm = q2.Artifact.load('HBM_Matrix_LCMS_1500/distance_matrix1500_jaccard_qiime2_noise_detect2_noblanks_HBM_R.qza')
j_stats = beta_group_significance(j_dm,q2mf_filtered.get_column('ATTRIBUTE_type'))
j_stats.visualization.save('HBM_Matrix_LCMS_1500/jacc_stats_storage.qzv')

In [ ]:
#beta diversity significance bray curtis 
bc_dm = q2.Artifact.load('core-metrics-10000/bray_curtis_distance_matrix.qza')
bc_stats = beta_group_significance(bc_dm,q2mf_filtered.get_column('host_common_name'))
bc_stats.visualization.save('core-metrics-10000/bc_stats_host.qzv')

In [ ]:
#beta diversity significance jaccard
j_dm = q2.Artifact.load('core-metrics-10000/jaccard_distance_matrix.qza')
j_stats = beta_group_significance(j_dm,q2mf_filtered.get_column('host_subject_id'))
j_stats.visualization.save('core-metrics-10000/jacc_stats_subject.qzv')

In [ ]:
#beta diversity significance bray curtis 
bc_dm = q2.Artifact.load('core-metrics-10000/bray_curtis_distance_matrix.qza')
bc_stats = beta_group_significance(bc_dm,q2mf_filtered.get_column('host_subject_id'))
bc_stats.visualization.save('core-metrics-10000/bc_stats_subject.qzv')